To only run one time at the beginning of the test (unless you want to change the current overlay) :

In [14]:
from pynq import Overlay
ol = Overlay("overlay/test_platform_kairos.bit")

To run only if you made change in the overlay :

In [15]:
ol.download()
# ol?

We define our SPI hardware object : AxiQspi.

In [16]:
from pynq import MMIO

SPI_regSpace = ol.ip_dict['axi_quad_spi_0']
IP_BASE_ADDRESS = SPI_regSpace['phys_addr']
IP_ADDRESS_RNGE = SPI_regSpace['addr_range']
AxiQspi = MMIO(IP_BASE_ADDRESS, IP_ADDRESS_RNGE)

Open the config file

In [17]:
with open("style.css","r") as file:
        css_file = file.read()
# print(css_file)

In [18]:
import yaml
with open("config.yaml","r") as file:
        config = yaml.safe_load(file)
        # print(config)

In [19]:
import yaml
with open("commands.yaml","r") as file:
        cmd_dict = yaml.safe_load(file)
        # print(config)

In [20]:
%run nb/messages.ipynb

# Config setup
activate_exceptions = config["LAUNCH_CONFIG"]["activate_exceptions"]
max_while_iterations = config["LAUNCH_CONFIG"]["max_while_iterations"]

# Launch config : choose to run all tests (with exceptions) or run a single test : see config.yaml
single_test_config = config["LAUNCH_CONFIG"]["single_test"]
all_test_config = config["LAUNCH_CONFIG"]["all_tests"]
if (single_test_config["activate"] == all_test_config["activate"]):
    test_config_error()

# Result file config : choose the result file extension to be html or txt : see config.yaml
activate_result_file_html = config["RESULT_FILE"]["html_config"]["activate"]
activate_result_file_txt = config["RESULT_FILE"]["txt_config"]["activate"]
activate_msg_prints = config["RESULT_FILE"]["print_msg"]["activate"]

In [21]:
%run nb/parse.ipynb
%run nb/package.ipynb
%run nb/functions.ipynb
%run nb/converter.ipynb
%run nb/spi/spi_driver.ipynb
%run nb/spi/spi_pkg.ipynb
%run nb/messages.ipynb
%run nb/translator.ipynb
        

import os 
#change to cnfg_master_spi
cnfg_master_spi(AxiQspi, clk_phase=0, clk_pol=0)
# ALL tests config :
if (all_test_config["activate"]) :
    all_test_files = os.listdir("./data")
    ignored_test_files = all_test_config['ignored_test_files']
    list_test_files = [f for f in all_test_files if f not in ignored_test_files]
    for test_file in list_test_files:
        test_file_name = test_file[:-4] # remove the txt extension
        test_file_path = 'data/' + test_file_name + '.txt'
        print(f"Currently sending test data from {test_file_name + '.txt'} ")
        
        if (activate_result_file_html):
            result_file_html = init_html_result_file(test_file_name, test_file_path, css_file)

        if (activate_result_file_txt):
            result_file_txt = init_txt_result_file(test_file_name, test_file_path)
        
        last_rd_data = [] # keep in memory the last read data for cmp commands
        with open(test_file_path,"r") as test_file_opened:
            parsed_file = run_parse_loop(test_file_opened, result_file_html, result_file_txt)
            number_cmd_test_file = len(parsed_file)
            for indice_command_from_test_file in range(1,number_cmd_test_file+1):
                parsed_line = parsed_file[indice_command_from_test_file]
                last_rd_data = command_disjunction(AxiQspi, result_file_html, result_file_txt, indice_command_from_test_file, parsed_line, last_rd_data)
                
        if (activate_result_file_html):
            end_html_result_file()

        if (activate_result_file_txt):
            end_txt_result_file()
                
        print(f"Finished sending test data from {test_file + '.txt'} ")
     
    
    
# SINGLE test config :
if (single_test_config["activate"]) :
    test_file_name = single_test_config['test_file'] 
    test_file_path = 'data/' + test_file_name + '.txt'
    print(f"Currently sending test data from {test_file_name + '.txt'} ")

    if (activate_result_file_html):
        result_file_html = init_html_result_file(test_file_name, test_file_path, css_file)

    if (activate_result_file_txt):
        result_file_txt = init_txt_result_file(test_file_name, test_file_path)
    
    last_rd_data = [] # keep in memory th  e last read data for cmp commands
    with open(test_file_path,"r") as test_file_opened: 
        parsed_file = run_parse_loop(test_file_opened, result_file_html, result_file_txt)
        number_cmd_test_file = len(parsed_file)
        for indice_command_from_test_file in range(1,number_cmd_test_file+1):
            parsed_line = parsed_file[indice_command_from_test_file]
            last_rd_data = command_disjunction(AxiQspi, result_file_html, result_file_txt, indice_command_from_test_file, parsed_line, last_rd_data)
            
    if (activate_result_file_html):
        end_html_result_file()

    if (activate_result_file_txt):
        end_txt_result_file()
            
    print(f"Finished sending test data from {test_file_name + '.txt'} ")
    

Configure device
Currently sending test data from test_mem1.txt 
REGWR 	 REGWR 0 654321FF CBA9871F   
 
REGWR (11) : B0 00 00 FF 21 43 65 1F 87 A9 CB  
 
Command # 1: Finished Writing  

REGRD 	 REGRD 0   
 
REGRD (3) : A0 00 00  
 
Command # 2: Finished Reading; read value = 0000000F  

REGWR 	 REGWR 0 452301FF 0   
 
REGWR (11) : B0 00 00 FF 01 23 45 00 00 00 00  
 
Command # 3: Finished Writing  

REGWR 	 REGWR 20 FFFFFFFF 22222222   
 
REGWR (11) : B0 20 00 FF FF FF FF 22 22 22 22  
 
Command # 4: Finished Writing  

REGRD 	 REGRD 20   
 
REGRD (3) : A0 20 00  
 
Command # 5: Finished Reading; read value = 22222222  

REGWR 	 REGWR 24 FFFFFFFF 00020002   
 
REGWR (11) : B0 24 00 FF FF FF FF 02 00 02 00  
 
Command # 6: Finished Writing  

REGRD 	 REGRD 24   
 
REGRD (3) : A0 24 00  
 
Command # 7: Finished Reading; read value = 00020002  

REGWR 	 REGWR 28 FFFFFFFF 00020000   
 
REGWR (11) : B0 28 00 FF FF FF FF 00 00 02 00  
 
Command # 8: Finished Writing  

REGRD 	 REGRD 28   
 

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
# Show clocks frequency
# from pynq import Clocks
# print(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')
# print(f'FCLK1: {Clocks.fclk1_mhz:.6f}MHz')